In [7]:
import gdown
import tarfile
import numpy as np
import cv2
from io import BytesIO
from skimage import io
from skimage.util import montage
import matplotlib.pyplot as plt
import os
import sys

def apply_logarithm(image):
    img_float = np.float32(image)
    log_image = np.log(1 + img_float)
    log_image = cv2.normalize(log_image, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(log_image)

def apply_exponential(image):
    img_float = np.float32(image)
    img_float /= 255.0
    exp_image = np.exp(img_float) - 1
    exp_image = cv2.normalize(exp_image, None, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(exp_image)

def apply_mean_filter(image, kernel_size=3):
    kernel = np.ones((kernel_size, kernel_size), np.float32) / (kernel_size ** 2)
    mean_filtered_image = cv2.filter2D(image, -1, kernel)
    return mean_filtered_image

def display_images(images, title):
    # Não há conversão para RGB aqui
    montage_image = montage([img[0] for img in sorted(images, key=lambda x: x[1])], grid_shape=(20, 4), padding_width=150, fill=(255, 255, 255), channel_axis=3)

    fig, ax = plt.subplots(figsize=(112, 84))
    ax.set_title(title, fontsize=16)
    ax.imshow(montage_image)
    ax.axis('off')

    img_height = montage_image.shape[0] // 20
    img_width = montage_image.shape[1] // 4

    for i, (img, name) in enumerate(sorted(images, key=lambda x: x[1])):
        row = i // 4
        col = i % 4
        x_pos = col * (img_width) + (img_width // 2)
        y_pos = row * (img_height - 7) + 100
        ax.text(x_pos, y_pos, name, color='black', fontsize=12, ha='center', va='bottom')

    plt.show()

def download_from_google_drive(file_id, output):
    url = f'https://drive.google.com/uc?id={file_id}'
    with open(os.devnull, 'w') as devnull:
        sys.stdout = devnull
        sys.stderr = devnull
        gdown.download(url, output, quiet=True)
        sys.stdout = sys.__stdout__
        sys.stderr = sys.__stderr__

def process_images_from_tar_gz(file_id, fruitFolders):
    output = 'file.tar.gz'
    download_from_google_drive(file_id, output)

    with open(output, 'rb') as f:
        file_like_object = BytesIO(f.read())

    with tarfile.open(fileobj=file_like_object, mode='r:gz') as tar:
        members = tar.getmembers()

        directories = sorted([m for m in members if m.isdir()], key=lambda x: x.name)

        for directory in directories:
            fruit = directory.name.split('/')[-1]
            if fruit not in fruitFolders:
                continue

            images = sorted([m for m in members if m.name.startswith(directory.name) and m.isfile()], key=lambda x: x.name)

            processed_images = []
            for img_member in images:
                file = tar.extractfile(img_member)
                if file:
                    image = io.imread(file)
                    if image is None:
                        continue

                    # Aplica as transformações nas imagens
                    logImage = apply_logarithm(image)
                    expImage = apply_exponential(image)
                    meanFilteredImage = apply_mean_filter(image)

                    # Define os nomes das imagens
                    imageName = img_member.name.split('/')[-1]
                    logImageName = f"log_{imageName}"
                    expImageName = f"exp_{imageName}"
                    meanImageName = f"mean_{imageName}"

                    # Adiciona as imagens originais e processadas para exibição
                    processed_images.append((image, imageName))
                    processed_images.append((logImage, logImageName))
                    processed_images.append((expImage, expImageName))
                    processed_images.append((meanFilteredImage, meanImageName))

            # Exibe as imagens usando montage
            display_images(processed_images[:80], title=f"{fruit.capitalize()}")

# ID do arquivo no Google Drive
file_id = '17yStUzdDOAlOXr4ZAghBqgEHXB-NTeqt'

# Lista de pastas a serem processadas
fruitFolders = ['apple', 'araticum', 'avocado', 'banana', 'butia', 'khaki', 'mango', 'orange', 'star_fruit', 'ugli_fruit']

process_images_from_tar_gz(file_id, fruitFolders)

Output hidden; open in https://colab.research.google.com to view.